In [ ]:
# Install our pip package in the current Jupyter kernel (DO NOT MODIFY)
# 安装模型，不要修改
import sys
!PYTHONWARNINGS=ignore {sys.executable} -m pip install --disable-pip-version-check cartpolev0dqn

In [ ]:
import cartpolev0dqn

# 简单参数（修改部分）
BATCH_SIZE = 4 # 批处理的大小
EPISODE = 1000 # 训练轮次

# 进阶参数（酌情修改）
GAMMA = 0.9 # GAMMA值 
INITIAL_EPSILON = 0.5 # 初始学习率
FINAL_EPSILON = 0.01 # 最终学习率
REPLAY_SIZE = 10000 # 经验回放集合大小

# 创建模型
model = cartpolev0dqn.model()

In [ ]:
# TRAINING
# 训练部分
model.train(GAMMA, INITIAL_EPSILON, FINAL_EPSILON, BATCH_SIZE, REPLAY_SIZE, EPISODE)

In [ ]:
# TEST
# 测试部分
model.test(isDisplay=True)